In [1]:
import pickle as pkl
import time
import os
import numpy as np
import sys
from music21 import instrument, note, stream, chord, duration
from RNNmodel import create_network
import matplotlib.pyplot as plt

2023-05-08 15:01:45.133857: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 15:01:48.950437: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
with open("dictionary.pkl", "rb") as input_file:
    dictionary = pkl.load(input_file)


durations_to_int = dictionary["durations_to_int"]
int_to_duration = dictionary["int_to_duration"]
notes_to_int = dictionary["notes_to_int"]
int_to_notes = dictionary["int_to_notes"]
phrases_to_int = dictionary["phrases_to_int"]
int_to_phrases = dictionary["int_to_phrases"]

n_notes = len(notes_to_int)
n_durations = len(durations_to_int)
n_phrases = len(phrases_to_int)

In [3]:
weights_folder = 'weights'
weights_file = 'weights.h5'
embed_size = 100
rnn_units = 256
use_attention = True

model, att_model = create_network(n_notes, n_durations, n_phrases, embed_size, rnn_units, use_attention)

# Load the weights to each node
weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)
model.summary()

2023-05-08 15:01:52.793320: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:01:53.522289: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:01:53.522464: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:01:53.534253: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-08 15:01:53.534338: I tensorflow/compile

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 100)    4800        ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 100)    1900        ['input_2[0][0]']                
                                                                                              

In [4]:
import pickle
piece = None
with open('../data/data_2/data_germany_1.pkl', 'rb') as handle:
    piece = pickle.load(handle)

In [5]:
notes_input = [ notes_to_int[note] for note in piece[0][0] ]
print(notes_input)

[16, 21, 21, 21, 21, 21, 23, 25, 25, 25, 28, 23, 46, 28, 21, 46, 16, 21, 21, 21, 21, 23, 25, 25, 25, 28, 23, 46, 28, 21, 46, 28, 33, 32, 30, 28, 26, 25, 46, 25, 25, 46, 30, 28, 26, 46, 25, 26, 46, 30, 28, 25, 46, 24, 25, 46, 28, 33, 32, 30, 28, 26, 25, 46, 24, 25, 46, 28, 28, 30, 30, 30, 32, 30, 32, 33, 33, 46]


In [6]:
durations_input = [ durations_to_int[duration] for duration in piece[0][1] ]
print(durations_input)

[7, 7, 7, 6, 2, 4, 4, 7, 7, 7, 7, 10, 7, 7, 10, 7, 7, 7, 7, 7, 4, 4, 7, 7, 7, 7, 10, 7, 7, 10, 7, 4, 4, 13, 10, 13, 10, 10, 7, 7, 10, 10, 10, 10, 10, 7, 7, 10, 10, 10, 10, 10, 7, 7, 10, 10, 10, 10, 13, 10, 13, 10, 10, 7, 7, 10, 1, 7, 7, 12, 7, 10, 10, 10, 10, 14, 10, 10]


In [7]:
phrases_input = piece[0][2]
print(phrases_input)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [8]:
max_extra_notes = len(notes_input) - 1
seq_len = 8
phrases_output = []
phrases_output.append(0) # after the first note (that is already in the initial sequence)

notes_input_sequence = [notes_input[0]] # the first note element of the sequence
durations_input_sequence = [durations_input[0]] # the first duration element of the sequence

if seq_len is not None:
    notes_input_sequence = [notes_to_int[129]] * (seq_len - len(notes_input_sequence)) + notes_input_sequence # the sequence length must be always equal seq_len, therefore we fill the vector with values 129 (the 'START' token) at the beginning 
    durations_input_sequence = [0] * (seq_len - len(durations_input_sequence)) + durations_input_sequence # the same for durations

In [9]:
print(notes_input_sequence)

[47, 47, 47, 47, 47, 47, 47, 16]


In [10]:
print(durations_input_sequence)

[0, 0, 0, 0, 0, 0, 0, 7]


In [11]:
for note_index in range(max_extra_notes):

    prediction_input = [
        np.array([notes_input_sequence])
        , np.array([durations_input_sequence])
       ]

    phrases_prediction = model.predict(prediction_input, verbose=0)
    print(notes_input_sequence)
    print(durations_input_sequence)
    print(phrases_prediction[0][0])
    print(phrases_prediction[0][1])
    
    phrases_output.append(0 if phrases_prediction[0][0] >= phrases_prediction[0][1] else 1)

    notes_input_sequence.append(notes_input[note_index])
    durations_input_sequence.append(durations_input[note_index])
    
    if len(notes_input_sequence) > seq_len:
        notes_input_sequence = notes_input_sequence[1:]
        durations_input_sequence = durations_input_sequence[1:]

2023-05-08 15:02:05.589657: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-08 15:02:05.591582: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-08 15:02:05.597550: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

[47, 47, 47, 47, 47, 47, 47, 16]
[0, 0, 0, 0, 0, 0, 0, 7]
0.99459344
0.0054065175
[47, 47, 47, 47, 47, 47, 16, 16]
[0, 0, 0, 0, 0, 0, 7, 7]
0.9681948
0.031805277
[47, 47, 47, 47, 47, 16, 16, 21]
[0, 0, 0, 0, 0, 7, 7, 7]
0.9995584
0.00044157315
[47, 47, 47, 47, 16, 16, 21, 21]
[0, 0, 0, 0, 7, 7, 7, 7]
0.99988294
0.00011707219
[47, 47, 47, 16, 16, 21, 21, 21]
[0, 0, 0, 7, 7, 7, 7, 6]
0.9995158
0.00048428145
[47, 47, 16, 16, 21, 21, 21, 21]
[0, 0, 7, 7, 7, 7, 6, 2]
0.99991393
8.607655e-05
[47, 16, 16, 21, 21, 21, 21, 21]
[0, 7, 7, 7, 7, 6, 2, 4]
0.999949
5.0970306e-05
[16, 16, 21, 21, 21, 21, 21, 23]
[7, 7, 7, 7, 6, 2, 4, 4]
0.99997973
2.022473e-05
[16, 21, 21, 21, 21, 21, 23, 25]
[7, 7, 7, 6, 2, 4, 4, 7]
0.99998
2.0016922e-05
[21, 21, 21, 21, 21, 23, 25, 25]
[7, 7, 6, 2, 4, 4, 7, 7]
0.9997056
0.00029432113
[21, 21, 21, 21, 23, 25, 25, 25]
[7, 6, 2, 4, 4, 7, 7, 7]
0.9999858
1.41357405e-05
[21, 21, 21, 23, 25, 25, 25, 28]
[6, 2, 4, 4, 7, 7, 7, 7]
0.999851
0.00014899184
[21, 21, 23, 25, 25,

In [12]:
print(phrases_output)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
print(phrases_input) # should be

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [14]:
for i in range(len(notes_input)):
    if phrases_output[i] == 1:
        k = i+1
        while k < len(notes_input) and k > 1:
            phrases_input[k-1] = 0
            phrases_input[k] = 1
            if notes_input[k] != 128:
                break

In [15]:
print(phrases_output)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
